In [2]:
import pandas as pd
import seaborn as sns
import networkx as nx
import numpy as np
import re
import matplotlib.pyplot as plt
import itertools
import sklearn.datasets
from sklearn.metrics.pairwise import euclidean_distances
import math
from sklearn.metrics.pairwise import rbf_kernel

In [5]:
#data = sklearn.datasets.load_iris()
#df = pd.DataFrame(data=data.data, columns=data.feature_names)
df = pd.read_csv("iris.csv", decimal=",", delimiter=";")

In [6]:
df

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [44]:
dfNumeric = df.select_dtypes(include=np.number)
dfNumeric

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [45]:
df.head(17)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [46]:
def euclideanDistance(source, target):
    totalSum = 0
    for i in range(len(source)):
        totalSum += pow(source[i] - target[i],2)
    return math.sqrt(totalSum)

def getDistanceMatrix(df):
    matrix = []
    for i in range(len(df)):
        matrix.append([])
    for i in range(len(df)):
        for j in range(len(df)):
            matrix[i].append(euclideanDistance(df.iloc[i],df.iloc[j]))
    return matrix


In [47]:
rbf_distances = rbf_kernel(dfNumeric, gamma=1)
rbf_distances

array([[1.00000000e+00, 7.48263568e-01, 7.71051586e-01, ...,
        2.30082059e-09, 4.03840951e-10, 3.59908895e-08],
       [7.48263568e-01, 1.00000000e+00, 9.13931185e-01, ...,
        1.62136087e-09, 2.15082380e-10, 3.22418674e-08],
       [7.71051586e-01, 9.13931185e-01, 1.00000000e+00, ...,
        3.65410404e-10, 6.16221335e-11, 9.42405852e-09],
       ...,
       [2.30082059e-09, 1.62136087e-09, 3.65410404e-10, ...,
        1.00000000e+00, 6.83861409e-01, 6.63650250e-01],
       [4.03840951e-10, 2.15082380e-10, 6.16221335e-11, ...,
        6.83861409e-01, 1.00000000e+00, 5.54327285e-01],
       [3.59908895e-08, 3.22418674e-08, 9.42405852e-09, ...,
        6.63650250e-01, 5.54327285e-01, 1.00000000e+00]])

In [48]:
# Similarity matrix -> Has weird values, bit different, perhaps I need to make my own distance matrix
distances = np.array(getDistanceMatrix(dfNumeric))
sigma = 1
distances = distances / -(2 * pow(sigma, 2))
distances = np.exp(distances)

In [49]:
def kNNAlgorithm(k, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
        for n in range(0, k):
            shortestDistance = -1
            shortestIndex = -1
            for j in range(0, len(distances)):
                if distances[i][j] >= shortestDistance and i != j: # I called it distance but its actually similarity, the higher the better
                    shortestDistance = distances[i][j]
                    shortestIndex = j
            distances[i][shortestIndex] = -1 # Set to inf distance thats already found to be closest
            if shortestIndex in edgeList.keys():
                if (i, shortestDistance) not in edgeList[shortestIndex]:
                    edgeList[i].append((shortestIndex, shortestDistance))
            else:
                edgeList[i].append((shortestIndex, shortestDistance))
    return edgeList

In [50]:
def gaussian_kernel(X, sigma):
    distances = np.sum((X[:, np.newaxis] - X) ** 2, axis=-1)
    kernel_matrix = np.exp(-distances / (2 * sigma**2))

    return kernel_matrix

distances = gaussian_kernel(dfNumeric.to_numpy(), 1)

In [51]:
#kNNResult = kNNAlgorithm(3, distances=np.copy(rbf_distances))
kNNResult = kNNAlgorithm(3, distances=np.copy(distances))
kNNResult

{0: [(17, 0.9950124791926823),
  (4, 0.9900498337491681),
  (39, 0.990049833749168)],
 1: [(45, 0.990049833749168),
  (12, 0.990049833749168),
  (37, 0.9851119396030626)],
 2: [(47, 0.990049833749168),
  (3, 0.9704455335485082),
  (45, 0.9656054162575665)],
 3: [(47, 0.990049833749168),
  (29, 0.9851119396030626),
  (30, 0.9753099120283326)],
 4: [(40, 0.9851119396030626), (17, 0.9851119396030626)],
 5: [(18, 0.9464851479534839),
  (10, 0.9417645335842487),
  (48, 0.9370674633774034)],
 6: [(47, 0.9753099120283327),
  (2, 0.9656054162575665),
  (11, 0.9559974818330998)],
 7: [(39, 0.9950124791926823),
  (49, 0.990049833749168),
  (0, 0.9851119396030626)],
 8: [(38, 0.9900498337491681), (3, 0.9559974818331), (42, 0.9512294245007139)],
 9: [(37, 1.0), (34, 1.0), (30, 0.9851119396030626)],
 10: [(48, 0.9950124791926822),
  (27, 0.9607894391523232),
  (36, 0.9559974818331)],
 11: [(29, 0.9753099120283327),
  (7, 0.9753099120283326),
  (26, 0.9607894391523232)],
 12: [(37, 0.985111939603062

In [52]:
def networkToDataframe(edgeList : dict):
    result = []
    i = 0
    for key, value in edgeList.items():
        for pair in value:
            result.append([])
            result[i].append(key)
            result[i].append(pair[0])
            result[i].append(pair[1])
            i += 1
    return result


In [58]:
def eRadius(epsilon, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
    for i in range(0, len(distances)):
        for j in range(0, len(distances)):
            if distances[i][j] > epsilon and j not in edgeList[i] and i != j: # If similarity is higher than 0.9 (epsilon)?
                if j in edgeList.keys():
                    if (i, distances[i][j]) not in edgeList[j]:
                        edgeList[i].append((j, distances[i][j]))
                #edgeList[i].append((j, distances[i][j]))
    return edgeList

In [54]:
dfKNN = pd.DataFrame(data=networkToDataframe(edgeList=kNNResult), columns=['from', 'to', 'similarity'])

In [55]:
dfKNN.to_csv("KNNAlgorithmResult.csv")
display(dfKNN)

,from,to,similarity
0,0,17,0.995012
1,0,4,0.990050
2,0,39,0.990050
3,1,45,0.990050
4,1,12,0.990050
...,...,...,...
309,145,147,0.937067
310,146,126,0.927743
311,148,110,0.856415
312,149,138,0.951229


In [59]:
resultERadius = eRadius(0.9, np.copy(rbf_distances))
resultERadius = eRadius(0.9, np.copy(distances))
print(resultERadius)
dfERadius = pd.DataFrame(data=networkToDataframe(edgeList=resultERadius), columns=['from', 'to', 'similarity'])
display(dfERadius)
dfERadius.to_csv("eRadiusResult.csv")

{0: [(4, 0.9900498337491681), (7, 0.9851119396030626), (10, 0.9323938199059479), (11, 0.9323938199059483), (17, 0.9950124791926823), (19, 0.9464851479534839), (20, 0.9093729344682312), (21, 0.9559974818330998), (26, 0.951229424500714), (27, 0.990049833749168), (28, 0.990049833749168), (31, 0.9277434863285526), (35, 0.9323938199059483), (36, 0.9185122844014573), (39, 0.990049833749168), (40, 0.9851119396030628), (43, 0.9003245225862656), (46, 0.9370674633774034), (48, 0.9559974818330998), (49, 0.9753099120283326)], 1: [(2, 0.9559974818330998), (3, 0.9464851479534836), (7, 0.9139311852712283), (9, 0.9851119396030626), (11, 0.9003245225862656), (12, 0.990049833749168), (25, 0.9753099120283326), (29, 0.9417645335842486), (30, 0.970445533548508), (34, 0.9851119396030626), (35, 0.9559974818331), (37, 0.9851119396030626), (39, 0.9003245225862658), (45, 0.990049833749168), (47, 0.9370674633774032), (49, 0.9512294245007141)], 2: [(3, 0.9704455335485082), (6, 0.9656054162575665), (7, 0.918512284

,from,to,similarity
0,0,4,0.990050
1,0,7,0.985112
2,0,10,0.932394
3,0,11,0.932394
4,0,17,0.995012
...,...,...,...
621,140,145,0.913931
622,141,145,0.970446
623,142,149,0.946485
624,143,144,0.951229


In [63]:
def kNNERadius(k, epsilon, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
        for n in range(0, k):
            shortestDistance = -1
            shortestIndex = -1
            for j in range(0, len(distances)):
                if distances[i][j] >= shortestDistance and i != j and distances[i][j] > epsilon: # I called it distance but its actually similarity, the higher the better
                    shortestDistance = distances[i][j]
                    shortestIndex = j
            distances[i][shortestIndex] = -1 # Set to inf distance thats already found to be closest
            if shortestIndex in edgeList.keys():
                if (i, shortestDistance) not in edgeList[shortestIndex]:
                    edgeList[i].append((shortestIndex, shortestDistance))
            else:
                if shortestIndex != -1:
                    edgeList[i].append((shortestIndex, shortestDistance))
    return edgeList

In [64]:
resultCombi = kNNERadius(3, 0.9, np.copy(rbf_distances))
resultCombi

{0: [(17, 0.99004983374917),
  (39, 0.9801986733067592),
  (28, 0.9801986733067592)],
 1: [(12, 0.9801986733067523),
  (45, 0.9801986733067453),
  (37, 0.9704455335485139)],
 2: [(47, 0.9801986733067523),
  (3, 0.9417645335842533),
  (12, 0.9323938199059479)],
 3: [(47, 0.9801986733067592),
  (29, 0.9704455335485002),
  (30, 0.9512294245007099)],
 4: [(0, 0.9801986733067592),
  (40, 0.9704455335485139),
  (17, 0.9704455335485139)],
 5: [],
 6: [(47, 0.9512294245007167),
  (2, 0.9323938199059347),
  (11, 0.9139311852712251)],
 7: [(39, 0.9900498337491771),
  (49, 0.9801986733067523),
  (0, 0.970445533548507)],
 8: [(38, 0.9801986733067523),
  (3, 0.9139311852712251),
  (42, 0.9048374180359519)],
 9: [(37, 1.0), (34, 1.0), (30, 0.9704455335485139)],
 10: [(48, 0.990049833749163),
  (27, 0.9231163463866242),
  (36, 0.9139311852712381)],
 11: [(7, 0.9512294245007167),
  (29, 0.9512294245007099),
  (26, 0.923116346386644)],
 12: [(37, 0.970445533548507), (34, 0.970445533548507)],
 13: [(38,

In [65]:
dfCombi = pd.DataFrame(data=networkToDataframe(edgeList=resultCombi), columns=['from', 'to', 'similarity'])
display(dfCombi)
dfCombi.to_csv("CombinationResult.csv")

,from,to,similarity
0,0,17,0.990050
1,0,39,0.980199
2,0,28,0.980199
3,1,12,0.980199
4,1,45,0.980199
...,...,...,...
158,136,148,0.941765
159,140,144,0.941765
160,141,145,0.941765
161,143,144,0.904837


In [24]:
distances

NameError: name 'distances' is not defined

In [ ]:
# TODO: Fix the distance, plot it in gelphi